In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from tests import TEST_NB, FAIL_NB
from nb_helpers.utils import *
from nb_helpers.wandb import *

In [ ]:
summary_nbs(path=TEST_NB)

CONSOLE.is_terminal(): False


Writing output to /Users/tcapelle/wandb/nb_helpers/tests/data/summary.csv

Reading 1 notebooks

┌───┬──────────────────────────┬────────────┬────────────────┬──────────────────────────────────────────────┬────────────┬───────┐
│ # │ nb name                  │ tracker    │ wandb features │ python libs                                  │ colab_cell │ colab │
├───┼──────────────────────────┼────────────┼────────────────┼──────────────────────────────────────────────┼────────────┼───────┤
│ 1 │ tests/data/test_nb.ipynb │ 0: tracker │                │ os, sys, pathlib, fastcore.basics, itertools │ 1          │ ]8;id=617469;https://colab.research.google.com/github/wandb/nb_helpers/blob/main/tests/data/test_nb.ipynb\open]8;;\  │
└───┴──────────────────────────┴────────────┴────────────────┴──────────────────────────────────────────────┴────────────┴───────┘

END!

## Search tracker and libs

In [ ]:
from rich import print
from rich.console import Console

from nb_helpers.utils import *
from nb_helpers.run import *
from nb_helpers.wandb import *

In [ ]:
repo = Path.cwd().parent
test_nb = repo/'tests/data/test_nb.ipynb'

In [ ]:
fname = test_nb
notebook = read_nb(fname)
print(fname)

/Users/tcapelle/wandb/nb_helpers/tests/data/test_nb.ipynb

In [ ]:
notebook["cells"][0]

{'cell_type': 'markdown',
 'metadata': {},
 'source': '# This is a Test NB\n<!--- @wandbcode{tracker} -->'}

In [ ]:
git_local_repo(fname)

Path('/Users/tcapelle/wandb/nb_helpers')

In [ ]:
get_colab_url(fname)

'https://colab.research.google.com/github/wandb/nb_helpers/blob/main/tests/data/test_nb.ipynb'

## Detect Pyton libs

In [ ]:
import re

In [ ]:
nb = read_nb(TEST_NB)

In [ ]:
def search_cells(nb, string: str = None, cell_type=CellType.code):
    "Search string in notebook cells, you can pass comma separated strings"
    strings = ifnone(string, "").replace(" ", "").split(",")
    cells = L()
    for cell in nb["cells"]:
        if cell["cell_type"] == cell_type:
            if any([string in cell["source"] for string in strings]):
                cells.append(cell["source"])
    return cells

In [ ]:
text_list = search_cells(nb, "import,from")
text_list

(#2) ['import os, sys\nimport logging as log\nfrom pathlib import Path\nfrom fastcore.basics import ifnone\n\ndef f(x): \n    return x','from itertools import chain']

In [ ]:

text_list = L([x.split("\n") for x in text_list]).concat()
text_list = [line for line in text_list if (("from" in line) or ("import" in line))]
text_list

['import os, sys',
 'import logging as log',
 'from pathlib import Path',
 'from fastcore.basics import ifnone',
 'from itertools import chain']

In [ ]:
strings = ['import tensorflow as tf','import tf.keras as K','import numpy','import sys, os']

In [ ]:
after_import_regex = re.compile(r'^import\s([^\.]*)', re.VERBOSE)
[after_import_regex.search(input).group(1) for input in strings]

['tensorflow as tf', 'tf', 'numpy', 'sys, os']

In [ ]:
before_as_regex = re.compile(r'([^\s]*?)\sas\s', re.VERBOSE)
before_as_regex.search('tensorflow as tf').group(1)

'tensorflow'

In [ ]:
between_from_import_regex = re.compile(r'^from\s(.*?)\simport', re.VERBOSE)
between_from_import_regex.search("from pathlib import Path").group(1)

'pathlib'

In [ ]:
strings = ['import tensorflow as tf','import tf.keras as K','import numpy','import sys, os', 'from fastcore.basics import ifnone']

In [ ]:
def _extract_libs(strings):
    "Automatically detect libraries imported in `strings`"

    after_import_regex = re.compile(r'^import\s([^\.]*)', re.VERBOSE)
    before_as_regex= re.compile(r'([^\s]*?)\sas\s', re.VERBOSE)
    between_from_import_regex=re.compile(r'^from\s(.*?)\simport', re.VERBOSE)


    def _search_with_regex(regex, string):
        res = regex.search(string)
        if res is not None:
            return res.group(1)
        else:
            return ""

    libs = []
    for string in strings:
        if "from" in string:
            string = _search_with_regex(between_from_import_regex, string).split(".")[0]
        else:
            string = _search_with_regex(after_import_regex, string)
            if "as" in string:
                string = _search_with_regex(before_as_regex, string)
        libs.append(string.replace(" ", "").split(","))
    return L(libs).concat()


In [ ]:
_extract_libs(strings)

(#6) ['tensorflow','tf','numpy','sys','os','fastcore']

In [ ]:
def detect_imported_libs(notebook):
    text_list = search_cells(notebook, "import,from")

    # format lines
    text_list = L([x.split("\n") for x in text_list]).concat()
    text_list = [line for line in text_list if (("from" in line) or ("import" in line))]
    
    return _extract_libs(text_list)


In [ ]:
fail_nb = read_nb(FAIL_NB)
detect_imported_libs(fail_nb)

(#0) []

In [ ]:
detect_imported_libs(read_nb(TEST_NB))


(#6) ['os','sys','logging','pathlib','fastcore','itertools']